In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC
from sklearn.metrics import classification_report
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer, SimpleImputer
from scipy.stats import loguniform, uniform
from pprint import pprint
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier

## subsampling: construct balanced data


In [22]:
PATH = '../data/raw'
df = pd.read_csv(os.path.join(PATH,'application_train.csv'))

mask = (df['TARGET']==1)
m = mask.sum()
df_balance = pd.concat([df[~mask].sample(m,random_state=1123),df[mask]], axis=0)

y = df_balance.pop("TARGET")
X = df_balance

X_tr, X_dev, y_tr, y_dev = train_test_split(X,y,test_size=0.2, random_state=1123)

ohe = OneHotEncoder(drop='if_binary')
scaler = StandardScaler()
model = HistGradientBoostingClassifier()

ct = make_column_transformer(
    (ohe, make_column_selector(dtype_exclude=np.number)),
    (scaler, make_column_selector(dtype_include=np.number)),
    remainder='drop',
)

pipe = make_pipeline(
    ct,model
)

pipe.fit(X_tr,y_tr)
y_pred = pipe.predict(X_dev)
print(classification_report(y_pred,y_dev))

              precision    recall  f1-score   support

           0       0.70      0.69      0.69      5076
           1       0.68      0.70      0.69      4854

    accuracy                           0.69      9930
   macro avg       0.69      0.69      0.69      9930
weighted avg       0.69      0.69      0.69      9930

